# **💳 카드 상품 추천 대화형AI 구현 프로젝트**
1. CSV파일 로드, Document객체로 만들기
2. 청킹
3. 임베딩
4. VectorDB에 저장
5. 검색
6. 프롬프트 작성
7. LLM모델 객체 생성

In [7]:
!pip install -U langchain-community

In [8]:
!pip install langchain-openai

In [9]:
from openai import OpenAI
from getpass import getpass
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

from langchain_openai import OpenAIEmbeddings

## 1. CSV 파일 로드, Document 객체 만들기
- CSVLoader 사용

In [12]:
%cd /content/drive/MyDrive/Colab Notebooks/SeSAC

[WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/drive/MyDrive/Colab Notebooks/SeSAC'
C:\Users\82108\study seoan


In [13]:
loaders = CSVLoader('data/debit_cards.csv', encoding = 'utf-8-sig')
doc = loaders.load()
doc

RuntimeError: Error loading data/debit_cards.csv

## 2. 청킹
- RecursiveCharacterTextSplitter 사용

In [6]:
# splitter 객체 생성
csv_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                             chunk_overlap = 50,
                                             separators = ["\n\n", "\n", " "])

In [ ]:
# loader와 splitter 실제로 적용
my_chunks = [] # CSV에서 잘라낸 청크들을 보관할 리스트

# load_and_split함수를 사용하면 반환되는 형식이 리스트이므로 extend 함수를 사용
my_chunks.extend(loader.load_and_split(csv_splitter))
print("number of chunks: ", len(my_chunks))
print(my_chunks)

number of chunks:  364
[Document(metadata={'source': 'data/debit_cards.csv', 'row': 0}, page_content='\ufeffcard_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 1}, page_content='\ufeffcard_name: 노리2 체크카드(KB Pay)\ncard_corp: KB국민카드\nbenefits: 커피, 모바일, 문화 10% 할인 뷰티, 편의점 5% 할인 KB Pay 2% 추가 할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 2}, page_content='\ufeffcard_name: KB 틴업 체크카드\ncard_corp: KB국민카드\nbenefits: 편의점 5% 할인 다이소, 올리브영 5% 할인 PC방, 모바일 5% 할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 3}, page_content='\ufeffcard_name: 모빌리언스카드\ncard_corp: KG모빌리언스\nbenefits: 온라인/오프라인 각각 1%/0.5% 적립 편의점, 커피, 외식 최대 8% 적립 KFC 10% 즉시할인'), Document(metadata={'source': 'data/debit_cards.csv', 'row': 4}, page_content='\ufeffcard_name: 더나은 체크카드\ncard_corp: MG새마을금고\nbenefits: 간편결제 20% 할인 온라인쇼핑 20% 할인 커피전문점 20% 할인'), Document(metadata={'source': 'data/deb

## ➡️ 청킹할 필요가 없더라!!^^;;

In [ ]:
MY_API_KEY = getpass("OpenAI API Key : ")

In [ ]:
client = OpenAI(api_key = MY_API_KEY)

In [ ]:
# 임베딩
my_embedding = OpenAIEmbeddings(model = "text-embedding-3-small",
                               api_key=MY_API_KEY)
my_embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7c2483d3a2d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7c2483c2ff90>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
# langchain연동 chroma DB 설치
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.3 MB/s e

In [ ]:
from langchain_chroma import Chroma

In [ ]:
my_directory = "/VectorStores" # DB가 저장될 폴더 경로를 지정

vectordb = Chroma.from_documents(documents = my_chunks, # 벡터화 시킬 청크 목록
                                embedding = my_embedding, # 임베딩 객체
                                persist_directory = my_directory) # 벡터 DB 저장 경로

In [ ]:
# 수집된 문서(청크) 개수 확인
vectordb._collection.count()

364

In [ ]:
vectordb._collection.get()

{'ids': ['aeaf6dc7-9452-4b35-a85b-79639d15cf45',
  'de27dd18-e1db-4d93-9d9e-9b534916331f',
  'b358d88d-bae1-47a0-84ae-6e3ff0480f12',
  'fc624b03-09fb-4263-8dd3-ad6efaf72ee2',
  '409ccbcd-b01d-426b-87dc-4f6a4634e4f6',
  'ab31407d-83f2-4365-942a-6d5d4ffedf00',
  '0dd9d7e7-50dc-4533-83cc-26aebe5e8685',
  'e6b8a58a-a394-4a36-a413-7ab184e8e9a8',
  '5ef88530-a5d9-48a1-94ce-8e58b57170c5',
  'f164c7e0-ab9e-4def-9184-35802d39d7d4',
  'd4bdd8e0-d8c3-4f90-82a2-e8400765c0cf',
  '4ab81bd5-a2fb-4804-9038-b8580088b8c8',
  '17c3e803-312c-48b9-9193-0ed8c0041328',
  'e4638445-9311-411c-a0a9-ac3395e01b53',
  '4fb42ca5-a1cc-42d3-911a-4d507168c3b5',
  'f5510924-7b80-4f67-b279-352bac8965cc',
  '6f25c23f-360e-41d2-bdb3-45714269b39e',
  '73f6c2d8-3761-414e-b2ef-d6d966a0b923',
  '54251e80-9d96-4583-bc0f-e783a0e4ff4e',
  '409ba70d-aa85-42fc-ad8a-c5de4610ae9c',
  'b48c3ff9-797b-4304-a915-4599a1530686',
  '55bf9c44-007e-4332-8cee-8a9aa76b0a85',
  'ccea0d2d-aef9-4884-806e-d8311c7e1444',
  '74a72ac1-10fc-4fc0-b389-

In [ ]:
question = "최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백해주는 카드가 뭐야?"
similar_docs = vectordb.similarity_search(question, k=3)
print("객체 수 : ", len(similar_docs)) # k값이 3이니까 객체 수는 3임.
print()
similar_docs

객체 수 :  3



[Document(id='aeaf6dc7-9452-4b35-a85b-79639d15cf45', metadata={'source': 'data/debit_cards.csv', 'row': 0}, page_content='\ufeffcard_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='0dd9d7e7-50dc-4533-83cc-26aebe5e8685', metadata={'row': 6, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: 토스뱅크 체크카드\ncard_corp: 토스뱅크\nbenefits: 일상 속 7가지 영역 월 최대 35,000원 캐시백 온라인 결제 7가지 영역 월 최대 14,000원 캐시백 국내 온·오프라인 어디서나 0.3% 캐시백'),
 Document(id='8a16cbd4-d242-4f5f-b875-c867614be3de', metadata={'source': 'data/debit_cards.csv', 'row': 243}, page_content='\ufeffcard_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립')]

In [ ]:
search_mmr = vectordb.max_marginal_relevance_search(question, fetch_k=6, k=3, lambda_mult=0.5)
search_mmr

[Document(id='aeaf6dc7-9452-4b35-a85b-79639d15cf45', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='8a16cbd4-d242-4f5f-b875-c867614be3de', metadata={'row': 243, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립'),
 Document(id='4a03af56-dbd6-4f71-8be7-3856fb17829f', metadata={'row': 117, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: go 캐시백 글로벌 체크카드\ncard_corp: 우체국\nbenefits: 해외전가맹점 7% 캐시백 간편결제 5% 캐시백 배달앱 5% 캐시백')]

In [ ]:
my_retriever = vectordb.as_retriever(search_kwargs={"k":3})

In [ ]:
question = "최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백해주는 카드가 뭐야?"

relevant_docs = my_retriever.invoke(question)
# my_retriever가 벡터DB에서 질문과 제일 가까운 문서 청크 3개를 찾아서 돌려줌.
relevant_docs

[Document(id='aeaf6dc7-9452-4b35-a85b-79639d15cf45', metadata={'source': 'data/debit_cards.csv', 'row': 0}, page_content='\ufeffcard_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='0dd9d7e7-50dc-4533-83cc-26aebe5e8685', metadata={'row': 6, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: 토스뱅크 체크카드\ncard_corp: 토스뱅크\nbenefits: 일상 속 7가지 영역 월 최대 35,000원 캐시백 온라인 결제 7가지 영역 월 최대 14,000원 캐시백 국내 온·오프라인 어디서나 0.3% 캐시백'),
 Document(id='8a16cbd4-d242-4f5f-b875-c867614be3de', metadata={'source': 'data/debit_cards.csv', 'row': 243}, page_content='\ufeffcard_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립')]

In [ ]:
# 검색기에 MMR검색도 적용 가능
# fetch_k=6 → 일단 6개 뽑아보고
# k=3 → 그중에서 최종적으로 3개만 선택
my_retriever_mmr = vectordb.as_retriever(search_type="mmr",
                                         search_kwargs={"fetch_k":6, "k":3, "lambda_mult":0.5}
                                        )

In [ ]:
relevant_docs_mmr = my_retriever_mmr.invoke(question)
relevant_docs_mmr

[Document(id='aeaf6dc7-9452-4b35-a85b-79639d15cf45', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
 Document(id='8a16cbd4-d242-4f5f-b875-c867614be3de', metadata={'row': 243, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립'),
 Document(id='4a03af56-dbd6-4f71-8be7-3856fb17829f', metadata={'source': 'data/debit_cards.csv', 'row': 117}, page_content='\ufeffcard_name: go 캐시백 글로벌 체크카드\ncard_corp: 우체국\nbenefits: 해외전가맹점 7% 캐시백 간편결제 5% 캐시백 배달앱 5% 캐시백')]

In [ ]:
!pip install pydantic

In [ ]:
# LLM 생성
chat_model = ChatOpenAI(model='gpt-3.5-turbo',
                        api_key=MY_API_KEY
                       )

In [ ]:
R_QA1 = RetrievalQA.from_chain_type(llm=chat_model,               # 답변해줄 LLM 지정
                                    chain_type="refine",          # 문서 처리 방식 선택
                                    retriever=my_retriever,       # 벡터DB 검색기
                                    return_source_documents=True  # 어떤 내용을 참조했는지 출력
                                   )

In [ ]:
question = "기술평가가 뭐야?"
llm_response = R_QA1.invoke(question) # "검색 + 답변"이 한 번에 실행됨.
llm_response

{'query': '최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백해주는 카드가 뭐야?',
 'result': '입력한 새로운 정보에 따라 정확한 답은 "하나로 전자카드 메가캐쉬백 더드림 체크카드"입니다. 생활필수업종에서 2만원당 200원 캐시백과 통신료 최대 5천원 캐시백, 그리고 CU편의점에서 2% 적립까지 혜택을 제공합니다.',
 'source_documents': [Document(id='aeaf6dc7-9452-4b35-a85b-79639d15cf45', metadata={'row': 0, 'source': 'data/debit_cards.csv'}, page_content='\ufeffcard_name: ONE 체크카드\ncard_corp: 케이뱅크\nbenefits: 최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백'),
  Document(id='0dd9d7e7-50dc-4533-83cc-26aebe5e8685', metadata={'source': 'data/debit_cards.csv', 'row': 6}, page_content='\ufeffcard_name: 토스뱅크 체크카드\ncard_corp: 토스뱅크\nbenefits: 일상 속 7가지 영역 월 최대 35,000원 캐시백 온라인 결제 7가지 영역 월 최대 14,000원 캐시백 국내 온·오프라인 어디서나 0.3% 캐시백'),
  Document(id='8a16cbd4-d242-4f5f-b875-c867614be3de', metadata={'source': 'data/debit_cards.csv', 'row': 243}, page_content='\ufeffcard_name: 하나로 전자카드 메가캐쉬백 더드림 체크카드\ncard_corp: 하나카드\nbenefits: 생활필수업종 2만원당 200원 캐시백 통신료 최대 5천원 캐시백 CU편의점 포인트 2% 적립')]}

In [ ]:
print(llm_response['query']) # 질문 출력
print(llm_response['result']) # Retriever가 찾아낸 문서 조각들을 기반으로 LLM이 생성한 답변

최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백해주는 카드가 뭐야?
입력한 새로운 정보에 따라 정확한 답은 "하나로 전자카드 메가캐쉬백 더드림 체크카드"입니다. 생활필수업종에서 2만원당 200원 캐시백과 통신료 최대 5천원 캐시백, 그리고 CU편의점에서 2% 적립까지 혜택을 제공합니다.


In [ ]:
# langchain과 연동된 cohere 라이브러리 설치
!pip install langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.3/295.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.9 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1


In [2]:
from langchain_cohere import CohereRerank

### user -> RAG -> prompting -> LLM -> inferencing
prompt에는 page_content를 전달


In [6]:
COHERE_API_KEY = getpass("Cohere API Key :")

TypeError: 'module' object is not callable. Did you mean: 'getpass.getpass(...)'?

In [ ]:
# CohereRerank객체 생성
my_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY,
                         model="rerank-v3.5",   # cohere에서 지원하는 reranking모델
                         top_n=3                # rerank모델이 가장 관련성이 높은 3개의 결과만 반환
                        )

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever

In [ ]:
compression_retriever = ContextualCompressionRetriever(base_compressor=my_rerank,
                                                       base_retriever=my_retriever
                                                      )

In [ ]:
R_QA2 = RetrievalQA.from_chain_type(llm=chat_model,
                                    chain_type="refine",
                                    retriever=compression_retriever   # 압축검색기 지정
                                   )

llm_response2 = R_QA2.invoke(question)
llm_response2

{'query': '최대 1.1% 무제한 캐시백 자주 쓰는 곳에서 5% 캐시백 3번 결제할 때마다 1,000원 무제한 캐시백해주는 카드가 뭐야?',
 'result': "위의 새로운 정보를 반영하여 답변을 수정하겠습니다.\n\n토스뱅크에서 발급하는 '토스뱅크 체크카드'는 일상 속 7가지 영역에서 월 최대 35,000원, 온라인 결제 7가지 영역에서 월 최대 14,000원의 캐시백 혜택을 제공하고 있습니다. 또한 국내 온·오프라인 어디서나 이용 시 0.3%의 캐시백 혜택도 받을 수 있습니다. 하나로 전자카드 메가캐쉬백 더드림 체크카드와는 다르게, 토스뱅크 체크카드는 일상 속의 다양한 영역에서 캐시백 혜택을 받을 수 있으며, 온라인과 오프라인 모두에서 적용됩니다."}

In [ ]:
question = """
너는 고객들에게 딱 맞는 체크카드를 추천해주는 은행원이야.
20대 취준생 고객이 있어. 자차가 없어서 지하철과 버스를 타고 이동하고,
식사는 주로 편의점에서 해결해. OTT 1~2개 구독하고 있어.
이 고객에게 알맞는 체크카드 3개를 추천해줘.
"""

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages =[{'role':'system','content':doc[0].page_content},
               {'role':'user','content':question}],
                temperature = 0
)
print(completion.choices[0].message.content)

20대 취준생 고객을 위한 알맞은 체크카드를 추천해드리겠습니다. 

1. **ONE 체크카드 (케이뱅크)**
   - 혜택: 최대 1.1% 무제한 캐시백, 자주 쓰는 곳에서 5% 캐시백, 3번 결제할 때마다 1,000원 무제한 캐시백
   - 이유: 편의점에서 식사를 해결하는 경우가 많으므로 자주 쓰는 곳에서 5% 캐시백 혜택이 유용할 것입니다. 또한, OTT 구독 결제 시에도 캐시백을 받을 수 있어 부담 없이 이용할 수 있습니다.

2. **페이북 체크카드 (카카오뱅크)**
   - 혜택: 1% 페이백, 편의점 할인 혜택
   - 이유: 편의점 할인 혜택이 있어 편의점에서의 소비 시 부가적인 혜택을 누릴 수 있습니다. 또한, OTT 구독 결제 시에도 1%의 페이백을 받을 수 있어 소액 결제에 유용합니다.

3. **제로페이 체크카드 (신한은행)**
   - 혜택: 최대 0.7% 캐시백, 제로페이 결제 시 수수료 면제
   - 이유: 제로페이 결제 시 수수료 면제 혜택이 있어, 지하철과 버스 요금 결제 시에도 혜택을 받을 수 있습니다. 또한, 적립된 캐시백은 신용카드나 체크카드로 전환 가능하여 다양한 용도로 활용할 수 있습니다.

이 세 가지 체크카드 중에서 ONE 체크카드가 가장 다양한 혜택을 제공하므로, 20대 취준생 고객님에게 가장 적합할 것으로 판단됩니다.


In [1]:
completion_base = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user","content":question}],
    temperature=0
)
print("=== BASE ===")
print(completion_base.choices[0].message.content)

# PROMPT 기반 호출
prompt = build_prompt("카드 데이터 예시: ...", question)  # context 대신 네 카드 데이터 넣기
completion_prompt = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user","content":prompt}],
    temperature=0
)
print("=== PROMPT ===")
print(completion_prompt.choices[0].message.content)

# RAG 기반 호출 (네가 이미 작성한 부분)
completion_rag = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system","content":doc[0].page_content},
        {"role":"user","content":question}
    ],
    temperature = 0
)
print("=== RAG ===")
print(completion_rag.choices[0].message.content)

NameError: name 'client' is not defined